In [1]:
import numpy as np
import pandas as pd
import reverse_geocoder as rg 
import pprint 
from datetime import datetime,time, date
import requests
import json

In [ ]:
df = pd.read_csv('yellow_tripdata_2016-01.csv')

In [ ]:
df.shape

In [4]:
df.head()
# df

,trip_id,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,payment_type,total_amount
0,1,2016-01-02 00:47:57,2016-01-02 01:15:36,1,18.6,-73.790237,40.644024,-73.974937,40.759392,1,66.34
1,2,2016-01-02 00:50:05,2016-01-02 01:17:33,1,17.7,-73.776772,40.645390,-73.971863,40.765717,1,63.80
2,3,2016-01-02 00:51:33,2016-01-02 01:21:09,1,20.2,-73.789970,40.646820,-73.959518,40.771900,2,58.34
3,4,2016-01-02 00:52:22,2016-01-02 01:20:11,1,19.8,-73.776764,40.645378,-73.954391,40.784168,2,58.34
4,5,2016-01-02 00:53:22,2016-01-02 01:26:52,1,20.9,-73.776665,40.645351,-73.974983,40.782787,1,68.34


In [5]:
df.tpep_pickup_datetime = df.tpep_pickup_datetime.apply(lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S"))
df.tpep_dropoff_datetime = df.tpep_dropoff_datetime.apply(lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S"))
df = df.sort_values(by = 'tpep_pickup_datetime')
df.head()

,trip_id,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,payment_type,total_amount
200,201,2016-01-01 00:02:31,2016-01-01 00:36:28,1,21.00,-73.788681,40.647346,-73.981880,40.773045,1,69.99
201,202,2016-01-01 00:02:59,2016-01-01 00:27:21,1,20.30,-73.777771,40.646736,-73.958557,40.763794,4,58.34
202,203,2016-01-01 00:04:31,2016-01-01 00:35:24,1,18.52,-73.789978,40.647137,-73.958107,40.802235,1,72.92
203,204,2016-01-01 00:13:55,2016-01-01 00:48:45,1,20.80,-73.782135,40.644505,-73.980103,40.788994,1,72.89
204,205,2016-01-01 00:22:16,2016-01-01 00:49:53,1,17.37,-73.776749,40.645969,-73.960487,40.767979,1,63.36


In [1]:
df = df.drop(columns = ['VendorID', 'store_and_fwd_flag', 'fare_amount', 'tip_amount', 'tolls_amount', 'extra', 'mta_tax', 'improvement_surcharge' ])

In [15]:
df.head()

,trip_id,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,payment_type,total_amount,pickup_date,pickup_time
200,201,2016-01-01 00:02:31,2016-01-01 00:36:28,1,21.00,-73.788681,40.647346,-73.981880,40.773045,1,69.99,2016-01-01,00:02:31
201,202,2016-01-01 00:02:59,2016-01-01 00:27:21,1,20.30,-73.777771,40.646736,-73.958557,40.763794,4,58.34,2016-01-01,00:02:59
202,203,2016-01-01 00:04:31,2016-01-01 00:35:24,1,18.52,-73.789978,40.647137,-73.958107,40.802235,1,72.92,2016-01-01,00:04:31
203,204,2016-01-01 00:13:55,2016-01-01 00:48:45,1,20.80,-73.782135,40.644505,-73.980103,40.788994,1,72.89,2016-01-01,00:13:55
204,205,2016-01-01 00:22:16,2016-01-01 00:49:53,1,17.37,-73.776749,40.645969,-73.960487,40.767979,1,63.36,2016-01-01,00:22:16


In [16]:
df['dropoff_lat_long_tuple'] = list(zip(df.dropoff_latitude, df.dropoff_longitude))
df.head()

,trip_id,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,payment_type,total_amount,pickup_date,pickup_time,dropoff_lat_long_tuple
200,201,2016-01-01 00:02:31,2016-01-01 00:36:28,1,21.00,-73.788681,40.647346,-73.981880,40.773045,1,69.99,2016-01-01,00:02:31,"(40.773044586181605, -73.9818801879883)"
201,202,2016-01-01 00:02:59,2016-01-01 00:27:21,1,20.30,-73.777771,40.646736,-73.958557,40.763794,4,58.34,2016-01-01,00:02:59,"(40.7637939453125, -73.95855712890629)"
202,203,2016-01-01 00:04:31,2016-01-01 00:35:24,1,18.52,-73.789978,40.647137,-73.958107,40.802235,1,72.92,2016-01-01,00:04:31,"(40.8022346496582, -73.9581069946289)"
203,204,2016-01-01 00:13:55,2016-01-01 00:48:45,1,20.80,-73.782135,40.644505,-73.980103,40.788994,1,72.89,2016-01-01,00:13:55,"(40.788993835449205, -73.9801025390625)"
204,205,2016-01-01 00:22:16,2016-01-01 00:49:53,1,17.37,-73.776749,40.645969,-73.960487,40.767979,1,63.36,2016-01-01,00:22:16,"(40.7679786682129, -73.9604873657227)"


In [2]:
df = df[(df.RatecodeID == 2) & (df.trip_distance > 0.1) & (df.pickup_longitude != 0) & (df.pickup_latitude != 0) & (df.dropoff_longitude != 0) & (df.dropoff_latitude != 0)]    #source hub = JFK, RatecodeID = 2
df.head()
# df

In [11]:
df.shape

(212431, 12)

In [18]:
neg_fare_index = list(df[df.total_amount < 0].index)
pos_fare_index = []

for index in neg_fare_index:
    pickup_time = df.loc[index,'tpep_pickup_datetime']
    drop_time = df.loc[index, 'tpep_dropoff_datetime']
    pickup_long = df.loc[index, 'pickup_longitude']
    pickup_lat = df.loc[index, 'pickup_latitude']
    dropoff_long = df.loc[index, 'dropoff_longitude']
    dropoff_lat = df.loc[index, 'dropoff_latitude']
    total_amount =  -df.loc[index, 'total_amount']
    
    
    
    req_index = list(df[(df.tpep_pickup_datetime == pickup_time) & (df.tpep_dropoff_datetime == drop_time) &
             (df.pickup_longitude == pickup_long) & (df.dropoff_longitude == dropoff_long) &
             (df.pickup_latitude == pickup_lat) & (df.dropoff_latitude == dropoff_lat) &
             (df.total_amount == total_amount)].index)
    pos_fare_index.extend(req_index)
    
remove_index = pos_fare_index + neg_fare_index





In [19]:
df = df[np.logical_not(df.index.isin(remove_index))]
df.shape

(47065, 14)

In [20]:
geo = rg.RGeocoder(mode=2, verbose=True)
results = geo.query(list(df.loc[:,'dropoff_lat_long_tuple']))
df['dropoff_city'] = [odict['name'] for odict in results]
df = df[df['dropoff_city'] == 'Manhattan']
df.shape

Loading formatted geocoded file...


(47065, 15)

In [21]:
df = df[df['passenger_count'] < 3]
df.shape

(47065, 15)

In [22]:
df = df.reset_index(drop = True)

In [23]:
df.head()

,trip_id,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,payment_type,total_amount,pickup_date,pickup_time,dropoff_lat_long_tuple,dropoff_city
0,201,2016-01-01 00:02:31,2016-01-01 00:36:28,1,21.00,-73.788681,40.647346,-73.981880,40.773045,1,69.99,2016-01-01,00:02:31,"(40.773044586181605, -73.9818801879883)",Manhattan
1,202,2016-01-01 00:02:59,2016-01-01 00:27:21,1,20.30,-73.777771,40.646736,-73.958557,40.763794,4,58.34,2016-01-01,00:02:59,"(40.7637939453125, -73.95855712890629)",Manhattan
2,203,2016-01-01 00:04:31,2016-01-01 00:35:24,1,18.52,-73.789978,40.647137,-73.958107,40.802235,1,72.92,2016-01-01,00:04:31,"(40.8022346496582, -73.9581069946289)",Manhattan
3,204,2016-01-01 00:13:55,2016-01-01 00:48:45,1,20.80,-73.782135,40.644505,-73.980103,40.788994,1,72.89,2016-01-01,00:13:55,"(40.788993835449205, -73.9801025390625)",Manhattan
4,205,2016-01-01 00:22:16,2016-01-01 00:49:53,1,17.37,-73.776749,40.645969,-73.960487,40.767979,1,63.36,2016-01-01,00:22:16,"(40.7679786682129, -73.9604873657227)",Manhattan
